## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Domino Data Lab Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. You need to make sure that these datasets are uploaded and present in the same directory as this notebook file, since we will be importing these files in Spark and create the following tables under the __`country_club`__ database using Spark SQL.

1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into Spark to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the Domino platform, but for the final deliverable, please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

# Checking Existence of Spark Environment Variables

Make sure your notebook is loaded using a PySpark Workspace. If you open up a regular Jupyter workspace the following variables might not exist

In [1]:
spark

In [2]:
sqlContext

### Run the following if you failed to open a notebook in the PySpark Workspace

This will work assuming you are using Spark in the cloud on domino or you might need to configure with your own spark instance if you are working offline

In [3]:
if 'sc' not in locals():
    from pyspark.context import SparkContext
    from pyspark.sql.context import SQLContext
    from pyspark.sql.session import SparkSession
    
    sc = SparkContext()
    sqlContext = SQLContext(sc)
    spark = SparkSession(sc)

# Create a utility function to run SQL commands

Instead of typing the same python functions repeatedly, we build a small function where you can just pass your query to get results.

- Remember we are using Spark SQL in PySpark
- We can't run multiple SQL statements in one go (no semi-colon ';' separated SQL statements)
- We can run multi-line SQL queries (but still has to be a single statement)

In [4]:
def run_sql(statement):
    try:
        result = sqlContext.sql(statement)
    except Exception as e:
        print(e.desc, '\n', e.stackTrace)
        return
    return result

# Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [5]:
run_sql('drop database if exists country_club cascade')
run_sql('create database country_club')
dbs = run_sql('show databases')
dbs.toPandas()

,databaseName
0,country_club
1,default


# Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you.

To get started, first make sure you have already uploaded the three CSV files and they are present in the same directory as the notebook.

Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [6]:
# File location and type
file_location_bookings = "./Bookings.csv"
file_location_facilities = "./Facilities.csv"
file_location_members = "./Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [7]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
 |-- bookid: integer (nullable = true)
 |-- facid: integer (nullable = true)
 |-- memid: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- slots: integer (nullable = true)

Facilities Schema
root
 |-- facid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- membercost: double (nullable = true)
 |-- guestcost: double (nullable = true)
 |-- initialoutlay: integer (nullable = true)
 |-- monthlymaintenance: integer (nullable = true)

Members Schema
root
 |-- memid: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- address: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- telephone: string (nullable = true)
 |-- recommendedby: integer (nullable = true)
 |-- joindate: timestamp (nullable = true)



# Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [8]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [9]:
run_sql('use country_club')
run_sql('REFRESH table bookings')
run_sql('REFRESH table facilities')
run_sql('REFRESH table members')
tbls = run_sql('show tables')
tbls.toPandas()

,database,tableName,isTemporary
0,country_club,bookings,False
1,country_club,facilities,False
2,country_club,members,False


# Test a sample SQL query

__Note:__ You can use multi-line SQL queries (but still a single statement) as follows

In [10]:
result = run_sql('''
                    SELECT * 
                    FROM bookings 
                    LIMIT 3
                 ''')
result.toPandas()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2


# Your Turn: Solve the following questions with Spark SQL

- Make use of the `run_sql(...)` function as seen in the previous example
- You can write multi-line SQL queries but it has to be a single statement (no use of semi-colons ';')
- Make use of the `toPandas()` function as depicted in the previous example to display the query results

#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [11]:
result = run_sql('''
                 select name  from facilities
                 where membercost != 0
                 
                 ''')
result.toPandas()

,name
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


####  Q2: How many facilities do not charge a fee to members?

In [17]:
result = run_sql('''
                   select count(*)
                   from facilities
                   where membercost=0
                 ''')
result.toPandas()

,count(1)
0,4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [14]:
result = run_sql('''
                   select facid,name,membercost,monthlymaintenance
                   from facilities where membercost!=0 and membercost <=(.2*monthlymaintenance)
                  ---select * from facilities limit(5)
                ''')
result.toPandas()

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [15]:
result = run_sql('''
                   select * from facilities 
                   where facid in (1,5)
                   
                ''')
result.toPandas()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25.0,8000,200
1,5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [35]:
result = run_sql('''
                  select name,
                  CASE when monthlymaintenance >= 100 THEN 'expensive' ELSE 'cheap' END AS monthlymaintenance
                  from facilities
                 ''')
result.toPandas()

,name,monthlymaintenance
0,Tennis Court 1,expensive
1,Tennis Court 2,expensive
2,Badminton Court,cheap
3,Table Tennis,cheap
4,Massage Room 1,expensive
5,Massage Room 2,expensive
6,Squash Court,cheap
7,Snooker Table,cheap
8,Pool Table,cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [46]:
result = run_sql('''
                  --select * from members  
                  select firstname,surname from members where joindate=(select max(joindate) from members)
                 ''')
result.toPandas()

,firstname,surname
0,Darren,Smith


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [65]:
result = run_sql('''
                   --select * from members
                   --select distinct name from facilities
                   select concat(firstname,surname,court) as answer
                   from (select distinct m.firstname,m.surname,f.name as court
                   from members as m,bookings as b, facilities f
                   where m.memid=b.memid
                   and b.facid=f.facid
                   and f.name in ('Tennis Court 1','Tennis Court 2')
                   order by m.firstname)
                   
                 ''')
result.toPandas()

,answer
0,AnneBakerTennis Court 1
1,AnneBakerTennis Court 2
2,BurtonTracyTennis Court 2
3,BurtonTracyTennis Court 1
4,CharlesOwenTennis Court 2
5,CharlesOwenTennis Court 1
6,DarrenSmithTennis Court 2
7,DavidPinkerTennis Court 1
8,DavidJonesTennis Court 1
9,DavidFarrellTennis Court 2


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [113]:
result = run_sql('''
                   select concat(m.firstname,m.surname,f.name,CASE WHEN m.memid = 0 THEN b.slots * f.guestcost 
                   ELSE b.slots*f.membercost END) as answer
                   from members as m,bookings as b, facilities f
                   where m.memid=b.memid
                   and b.facid=f.facid
                   and b.starttime like '2012-09-14%'
                   and (CASE WHEN m.memid = 0 THEN b.slots * f.guestcost 
                   ELSE b.slots*f.membercost END) >30
                   order by CASE WHEN m.memid = 0 THEN b.slots * f.guestcost 
                   ELSE b.slots*f.membercost END
                 ''')
result.toPandas()

,answer
0,GUESTGUESTSquash Court35.0
1,GUESTGUESTSquash Court35.0
2,JemimaFarrellMassage Room 139.6
3,GUESTGUESTSquash Court70.0
4,GUESTGUESTTennis Court 275.0
5,GUESTGUESTTennis Court 175.0
6,GUESTGUESTTennis Court 175.0
7,GUESTGUESTTennis Court 2150.0
8,GUESTGUESTMassage Room 1160.0
9,GUESTGUESTMassage Room 1160.0


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [112]:
result = run_sql('''
                 SELECT concat(*) as answer
FROM (
    SELECT CONCAT( firstname,  ' ', surname ) AS membername, f.name, 
    CASE WHEN m.memid =0 THEN slots * guestcost
    ELSE slots * membercost
    END AS daycost
FROM  Bookings AS b,members as m,facilities as f
where b.memid = m.memid AND b.starttime LIKE  '2012-09-14%'
and b.facid = f.facid
     ) AS daycosts
WHERE daycost >30
order by daycost
                  
                 ''')

result.toPandas()

,answer
0,GUEST GUESTSquash Court35.0
1,GUEST GUESTSquash Court35.0
2,Jemima FarrellMassage Room 139.6
3,GUEST GUESTSquash Court70.0
4,GUEST GUESTTennis Court 175.0
5,GUEST GUESTTennis Court 175.0
6,GUEST GUESTTennis Court 275.0
7,GUEST GUESTTennis Court 2150.0
8,GUEST GUESTMassage Room 1160.0
9,GUEST GUESTMassage Room 1160.0


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [73]:

result = run_sql('''
                  SELECT f.name, SUM( 
      CASE WHEN m.memid =0 THEN slots * guestcost
      ELSE slots * membercost
      END ) AS revenue
FROM  Bookings AS b,members AS m,facilities AS f
where b.memid = m.memid
and b.facid = f.facid
GROUP BY f.name
having revenue<1000
ORDER BY revenue DESC 

                   
                 ''')
result.toPandas()

,name,revenue
0,Pool Table,270.0
1,Snooker Table,240.0
2,Table Tennis,180.0
